## Pipeline Parameters

In [0]:
dbutils.widgets.text("bronze_table_name", 'kdayno_bronze_reddit_top_posts')
dbutils.widgets.text("silver_table_name", 'kdayno_silver_reddit_top_posts_sentiment')

source_bronze_table_name = dbutils.widgets.get("bronze_table_name")
target_silver_table_name = dbutils.widgets.get("silver_table_name")

In [0]:
# Standard library imports
import os

# Third-party library imports
from dotenv import load_dotenv
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline
from pyspark.sql.functions import udf, col
from pyspark.sql.types import StringType, StructType, StructField, FloatType

load_dotenv()

In [0]:
catalog_name = os.getenv('DATABRICKS_CATALOG_NAME')
schema_name = os.getenv('DATABRICKS_SCHEMA_NAME')

In [0]:
# Load the FinBERT model and tokenizer
model_name = "ProsusAI/finbert"
model = AutoModelForSequenceClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)
classifier = pipeline('text-classification', model=model, tokenizer=tokenizer)

# Create a UDF for sentiment analysis
@udf(returnType=StructType([
    StructField("sentiment", StringType(), True),
    StructField("sentiment_score", FloatType(), True)
]))

def sentiment_analysis_udf(text):
    result = classifier(text)[0]
    return (result['label'], float(result['score']))

## Sentiment Analysis of Reddit Posts

In [0]:
# Apply the UDF and generate sentiment analysis results for posts
reddit_posts_df = spark.table(f"{catalog_name}.{schema_name}.{source_bronze_table_name}")

reddit_posts_sentiment_df = reddit_posts_df.withColumn("post_title_sentiment_result", sentiment_analysis_udf(col('post_title')))
reddit_posts_sentiment_df = reddit_posts_sentiment_df.select("*", "post_title_sentiment_result.*")

In [0]:
(reddit_posts_sentiment_df.write.format("delta")
                                .mode("overwrite")
                                .partitionBy('subreddit')
                                .saveAsTable(f"{catalog_name}.{schema_name}.{target_silver_table_name}"))